In [1]:
# section references are to:
# MISO Energy and Operating Reserve Markets Business Practices Manual, BPM-002-r19

from IPython.display import display

from io import StringIO
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
bid_file = '20201007_bids_cb.csv'
bid_df = pd.read_csv(bid_file)
offer_file = '20201007_da_co.csv'
offer_df = pd.read_csv(offer_file)

In [3]:
ts = '10/07/2020 00:00:00'
bid_df = bid_df.loc[
    (bid_df['Date/Time Beginning (EST)'] == ts)
    & ~bid_df['Region'].isnull()
]
offer_df = offer_df.loc[
    (offer_df['Date/Time Beginning (EST)'] == ts)
    & (offer_df['Unit Available Flag'] > 0)
]

In [4]:
bid_df.groupby('Type of Bid').head(1)

,Region,Market Participant Code,Date/Time Beginning (EST),Date/Time End (EST),MW,LMP,Type of Bid,Bid ID,PRICE1,MW1,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9
0,North,122062236,10/07/2020 00:00:00,10/07/2020 01:00:00,6.0,9.58,F,122073561,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,Central,122062261,10/07/2020 00:00:00,10/07/2020 01:00:00,0.0,15.00,D,981837552,15.0,205.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,North,122062269,10/07/2020 00:00:00,10/07/2020 01:00:00,15.0,7.99,I,3936522770,5.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17376,Central,122062313,10/07/2020 00:00:00,10/07/2020 01:00:00,155.8,10.36,P,450524077,190.0,155.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# keep fixed and price-sensitive bids (§4.3.2): throw away virtual bids for now
bid_df = bid_df.loc[bid_df['Type of Bid'].isin(['F', 'P'])]

In [6]:
# §4.3.2 gives the Energy Offer Hard Price Cap as 2000
MAX_PRICE = 2000

# §4.3.1 fixed price bids can pay up to the maximum price
bid_df.loc[bid_df['Type of Bid'] == 'F', 'PRICE1'] = MAX_PRICE

In [7]:
# §4.3.2: bids are submitted in blocks, in any order, but:
# "will appear sorted in descending price order, starting with the highest priced block"

def parse_bid_curve(row):
    bids = (
        pd.DataFrame({
            'price': [row['PRICE%d' % i] for i in range(1, 10)],
            'volume': [row['MW%d' % i] for i in range(1, 10)]},
            index=range(1, 10)
        )
        .dropna(how='any', axis='rows')
    )
    # ignore negative bids for now
    bids = (bids
        .loc[bids['price'] >= 0.]
        .reset_index()
    )
    return bids

def validate_bid(row):
    bids = parse_bid_curve(row)
    return len(bids) > 0


In [8]:
valid_bid_df = bid_df.loc[bid_df.apply(validate_bid, axis='columns')]

In [9]:
valid_bid_df.groupby('Date/Time Beginning (EST)').count()

,Region,Market Participant Code,Date/Time End (EST),MW,LMP,Type of Bid,Bid ID,PRICE1,MW1,PRICE2,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9
Date/Time Beginning (EST),,,,,,,,,,,,,,,,,,,,,
10/07/2020 00:00:00,296,296,296,296,296,296,296,296,296,4,...,1,1,1,1,1,1,1,1,0,0


In [10]:
def parse_offer_curve(row):
    offers = (
        pd.DataFrame({
            'price': [row['Price%d' % i] for i in range(1, 10)],
            'volume': [row['MW%d' % i] for i in range(1, 10)]},
            index=range(1, 10)
        )
        .dropna(how='any', axis='rows')
    )
    # ignore negative capacity for now
    offers = (offers
        .loc[offers['volume'] > 0.]
        .sort_values('price')
        .drop_duplicates('price', keep='last')
        .reset_index()
    )
    return offers

def validate_offer(row):
    offers = parse_offer_curve(row)
    if len(offers) == 0:
        return False
    elif offers.price.is_monotonic_increasing and offers.volume.is_monotonic_increasing:
        return True
    else:
        raise Exception(row)

In [11]:
valid_offer_df = offer_df.loc[offer_df.apply(validate_offer, axis='columns')]

In [12]:
valid_offer_df.groupby('Date/Time Beginning (EST)').count()

,Region,Unit Code,Date/Time End (EST),Economic Max,Economic Min,Emergency Max,Emergency Min,Economic Flag,Emergency Flag,Must Run Flag,...,MW6,Price7,MW7,Price8,MW8,Price9,MW9,Price10,MW10,Slope
Date/Time Beginning (EST),,,,,,,,,,,,,,,,,,,,,
10/07/2020 00:00:00,913,913,913,913,913,913,913,913,913,913,...,233,200,200,185,185,175,175,152,152,913


In [13]:
ts = valid_offer_df.query('`Date/Time Beginning (EST)` == "10/07/2020 11:00:00"')
ts.loc[~ts['MW5'].isnull()].head()

,Region,Unit Code,Date/Time Beginning (EST),Date/Time End (EST),Economic Max,Economic Min,Emergency Max,Emergency Min,Economic Flag,Emergency Flag,...,MW6,Price7,MW7,Price8,MW8,Price9,MW9,Price10,MW10,Slope


In [14]:
unpacked_offers = []
for row_idx, offer_row in valid_offer_df.iterrows():
# for row_idx, offer_row in valid_offer_df.query('`Unit Code` == "A5599"').iterrows():
    offer_curve = parse_offer_curve(offer_row)
    unit = offer_row['Unit Code']
    node = offer_row['Region']
    if len(offer_curve) == 1:
        unpacked_offers.append([unit, node, offer_curve.volume.iloc[0], offer_curve.price.iloc[0]])
        continue
    offer_curve['volume'] = offer_curve['volume'].diff().combine_first(offer_curve['volume'])
    for offer_idx, offer in offer_curve.iterrows():
        unpacked_offers.append(['%s_%d' % (unit, offer_idx), node, offer.volume, offer.price])
unpacked_offers = pd.DataFrame(unpacked_offers, 
                               columns=['id', 'node', 'capacity (MW)', 'offer ($/MW)'])
unpacked_offers.head()

,id,node,capacity (MW),offer ($/MW)
0,A3153_0,North,1.0,8.16
1,A3153_1,North,245.0,9.54
2,A3153_2,North,1.0,14.27
3,A3153_3,North,72.0,14.28
4,A3167_0,Central,330.0,8.74


In [15]:
unpacked_offers.to_csv(offer_file.replace('.csv', '.processed.csv'), index=False)

In [16]:
unpacked_bids = []
for row_idx, bid_row in valid_bid_df.iterrows():
    bid_curve = parse_bid_curve(bid_row).sort_values('volume')
    unit = bid_row['Bid ID']
    node = bid_row['Region']
    if len(bid_curve) == 1:
        unpacked_bids.append([unit, node, bid_curve.volume.iloc[0], bid_curve.price.iloc[0]])
        continue
    for bid_idx, bid in bid_curve.iterrows():
        unpacked_bids.append(['%s_%d' % (unit, bid_idx), node, bid.volume, bid.price])
unpacked_bids = pd.DataFrame(unpacked_bids, 
                             columns=['id', 'node', 'demand (MW)', 'bid ($/MW)'])
unpacked_bids.tail()

,id,node,demand (MW),bid ($/MW)
304,1656071307,South,31.6,2000.0
305,1656071307,South,31.6,2000.0
306,2791533958,South,4.5,2000.0
307,2791533958,South,4.5,2000.0
308,3554572649,Central,56.3,2000.0


In [17]:
unpacked_bids.to_csv(bid_file.replace('.csv', '.processed.csv'), index=False)

In [18]:
unpacked_offers['capacity (MW)'].sum()

145833.2

In [19]:
unpacked_bids['demand (MW)'].sum()

57460.899999999994